# Neural Horizon MPC tutorial

In [ ]:
import os
import logging
import numpy as np

from itertools import product

# logging.basicConfig(level=logging.DEBUG)

In [ ]:
from src.data_generation_acados import AMPC_dataset_gen
from src.parameters import AMPC_Param, get_dataset_name
from src.inverted_pendulum_acados import acados_model_RK4, nonlinear_pend_dynamics_acados

In [ ]:
MPC_DATASETS_DIR = os.path.abspath(os.path.join('Results', 'MPC_data_gen'))

## Set initial parameter values

In [ ]:
MPC_PARAM_DICT = {
    'T_sim': 5, # length of the closed-loop simulation (in seconds)
}
NUM_SAMPLES = 5_000


DATASET_VERSIONS = (9,)                 # for training and testing e.g.
DATASET_HORIZONS = (30, )

In [ ]:
# qp_solver
# ---------
#       'FULL_CONDENSING_QPOASES', 'FULL_CONDENSING_HPIPM', 'FULL_CONDENSING_DAQP',
#       'PARTIAL_CONDENSING_HPIPM', 'PARTIAL_CONDENSING_QPDUNES', 'PARTIAL_CONDENSING_OSQP'
# hessian_approx
# --------------
#       'GAUSS_NEWTON', 'EXACT'
# integrator_type
# ---------------
#       'ERK', 'IRK', 'DISCRETE', 'LIFTED_IRK', 'GNSF'
# nlp_solver_type
# ---------------
#       'SQP_RTI', 'SQP'
# regularize_method
# -----------------
#       'NO_REGULARIZE', 'MIRROR', 'PROJECT', 'PROJECT_REDUC_HESS', 'CONVEXIFY'
# hpipm_mode
# ----------
#       'BALANCE', 'SPEED_ABS', 'SPEED', 'ROBUST'
# collocation_type
# ----------------
#       'GAUSS_RADAU_IIA', 'GAUSS_LEGENDRE'
# globalization
# -------------
#       'FIXED_STEP', 'MERIT_BACKTRACKING'
# nlp_solver_tol_stat
# -------------------
#       Default: 1e-6
# nlp_solver_max_iter
# -------------------
#       Default: 100
#
#############################################
# use_iter_rti_impl
# -----------------
#       Default = False
# use_initial_guesses
# -----------------
#       Default = False
# max_rti_iters
# -----------------
#       Default = 10
# rti_tol
# -----------------
#       Default = 1e-4


ACADOS_OPTIONS = {
    f'RTI_PCHPIPM_DISCRETE': dict(
        qp_solver='PARTIAL_CONDENSING_HPIPM', 
        integrator_type='DISCRETE', 
        nlp_solver_type='SQP_RTI',
        use_iter_rti_impl=True,
        use_initial_guesses=True,
        rti_tol=1e-6
    ),
    # f'SQP_PCHPIPM_DISCRETE': dict(
    #     qp_solver='PARTIAL_CONDENSING_HPIPM', 
    #     integrator_type='DISCRETE', 
    #     nlp_solver_type='SQP',
    #     use_initial_guesses=True
    # ),
    # f'ASRTID_FCH': dict(
    #     qp_solver='FULL_CONDENSING_HPIPM', 
    #     integrator_type='DISCRETE', 
    #     nlp_solver_type='SQP_RTI',
    #     as_rti_iter=3,
    #     as_rti_level=3,
    #     nlp_solver_tol_stat=1e-6,
    #     nlp_solver_max_iter=3,
    # )
    # f'RTI_PCHPIPM_ROBUST_DISCRETE': dict(
    #     qp_solver='PARTIAL_CONDENSING_HPIPM', 
    #     integrator_type='DISCRETE', 
    #     nlp_solver_type='SQP_RTI',
    #     hpipm_mode='ROBUST',
    # ),
}

## Generating training dataset from given setups

In [ ]:
for dataset_horizon, dataset_version, (acados_name, solver_options) in \
    product(DATASET_HORIZONS, DATASET_VERSIONS, ACADOS_OPTIONS.items()):
    
    print('#' + '=' * 80)
    print(f'{NUM_SAMPLES} training data samples for a horizon of {dataset_horizon} -> Version {dataset_version}')
    print(f'Acados options: {acados_name}')

    ampc_param = AMPC_Param(N_MPC=dataset_horizon, **MPC_PARAM_DICT)
    model = acados_model_RK4(nonlinear_pend_dynamics_acados(ampc_param), ampc_param)
    sampler = AMPC_dataset_gen(
        model,
        ampc_param, 
        solver_options=solver_options,
        n_samples=NUM_SAMPLES,
        chance_reset_sample=0.25,
        init_scale=np.array([.75, .15, .25, .25])
    )
    gen_file = sampler.generate_data(
        filename=get_dataset_name(dataset_horizon, NUM_SAMPLES, acados_name, dataset_version),
        filedir=MPC_DATASETS_DIR
    )
    sampler.MPC.cleanup()
    del sampler.MPC